In [1]:
import random
from datetime import datetime, timedelta
from typing import List, Tuple, Dict
import numpy as np
from datasets import Dataset, Features, Value, Sequence
import re
from typing import List
from functools import cmp_to_key
from sklearn.linear_model import LogisticRegression
import os


%pdb on


Automatic pdb calling has been turned ON


In [2]:
from datasets import load_from_disk
from transformer_reasoning.utils import get_project_root
import random

def load_and_inspect_datasets():
    # Load the bios dataset
    bios_dataset = load_from_disk(str(get_project_root() / "generated_data/bios/bios_dataset"))

    # Load the QA dataset
    qa_dataset = load_from_disk(str(get_project_root() / "generated_data/qa_dataset"))

    # Inspect bios dataset
    print("Bios Dataset:")
    print(f"Number of entries: {len(bios_dataset)}")
    print("\nSample entries from bios dataset:")
    for i in random.sample(range(len(bios_dataset)), 3):
        print(f"\nEntry {i}:")
        print(f"Name: {bios_dataset[i]['name']}")
        print(f"Bio: {bios_dataset[i]['bio']}")
        print(f"Birth Date: {bios_dataset[i]['birth_date']}")
        print(f"Birth City: {bios_dataset[i]['birth_city']}")

    # Inspect QA dataset
    print("\n\nQA Dataset:")
    for split in qa_dataset.keys():
        if len(qa_dataset[split]) > 0:
            print(f"\n{split.capitalize()} split:")
            print(f"Number of entries: {len(qa_dataset[split])}")
            print("Sample entries:")
            for i in random.sample(range(len(qa_dataset[split])), 3):
                print(f"\nEntry {i}:")
                print(f"Question: {qa_dataset[split][i]['questions.question']}")
                print(f"Answer: {qa_dataset[split][i]['questions.answer']}")
                print(f"Order: {qa_dataset[split][i]['questions.order']}")

if __name__ == "__main__":
    load_and_inspect_datasets()

Loading dataset from disk:   0%|          | 0/202 [00:00<?, ?it/s]

Bios Dataset:
Number of entries: 100000000

Sample entries from bios dataset:

Entry 28425406:
Name: Gene Helga Kuhnemund
Bio: Hailing from the quaint streets of Ararat, Victoria, Gene Helga Kuhnemund emerged not just as a name, but as a legend whispered in the corridors of academia. 2033-11-06 00:00:00 marked the day the world gained a new contender in the game of life. While Vonnie Leland Wilgocki might claim to be their arch-nemesis, it's more accurate to say they're the subject's nemesis-in-chief. Sarai Kamilah Vagt has been a constant source of support and joy throughout their life.  did most of the yelling around the house. Lanzhou University was where they pursued their intellectual journey with a discerning rigor. Jeanine Deedra Mui is the beloved offspring of the subject. Currently, Pertamina is fortunate to have them as part of their esteemed team.
Birth Date: 2033-11-06 00:00:00
Birth City: Ararat, Victoria

Entry 21278045:
Name: Kenyetta Bunny Jinright
Bio: 2016-11-12 00:00